# Importing stream surveys with the StreamSurvey class

## Basics

Most geomorphic surveys are conducted using either a total station or laser level. This data is often stored or transcribed into a tabular format where each row indicates the number, northing, easting, elevation and description of each point (or shot) that was collected. An example of such data can be found [here](https://github.com/rsjones94/pyfluv/blob/master/pyfluv/Data/wpr_myr5_survey_adjusted.csv) (this data is included with pyfluv). Because breaking down this data can be tedious, especially for large datasets, pyfluv includes a StreamSurvey class which parses this data and exports CrossSection and Profile objects.

In [2]:
import pyfluv
survey = pyfluv.piney_survey()

The above snippet reads in the previously linked tabular survey and returns a StreamSurvey objects. A StreamSurvey object can be initialized with `CrossSection('path/to/your/file.csv')`. By default the csv must contain columns headed with `Name`, `Northing`, `Easting`, and `Description`. Other columns are permitted but will be ignored. The `Name` column should contain integers that generally indicate what order the shots were taken in, though the column itself does not need to be ordered. The StreamSurvey object will use the `Description` column to group profiles and cross sections together and decide what morphological feature a given shot corresponds to. Entries must follow the following format:

`[morphtype][name]-[indicator1]-[indicator2]-...-[indicatorN]_[comment]`

Let's take a look at some examples.

In [27]:
survey.data.iloc[2]

Name                      5
Northing            928.442
Easting             952.319
Elevation           665.254
Description    proFBSUP-bri
Name: 2, dtype: object

The survey data is stored as a `pandas` dataframe so we index into it using `iloc`, and the description is `proFBSUP-bri`. You can check how the StreamSurvey object will parse the descriptions:

In [6]:
print(survey.keywords)

{'Profile': 'pro', 'Thalweg': 'thw', 'Riffle': 'ri', 'Run': 'ru', 'Pool': 'po', 'Glide': 'gl', 'Water Surface': 'ws', 'Bankfull': 'bkf', 'Top of Bank': 'tob', 'Cross Section': 'xs', 'Structure': 'str', 'breakChar': '-', 'commentChar': '_'}


The keywords attribute is a dictionary that relates survey shorthands to stream features. StreamSurvey will use this dictionary to figure out what a shot represents. The `breakChar` key indicates that a hyphen (`-`) will be used to split the survey description. Characters before the first break will be evaluated for two things: whether or not it corresponds to a profile or a cross section and what reach or cross section group it belongs to. In this case, `proFBSUP` indicates a profile with the name FBSUP. All segments after the first break are evaluated for morphological features. The second segment is `bri` which contains `ri` and so is considered a riffle. The keywords dictionary also contains a `commentChar` key, which tells us that comments are indicated with an underscore (`_`). Any characters after a comment character are not used during parsing.

In [26]:
survey.data.iloc[3]

Name                     6
Northing           928.298
Easting            952.014
Elevation          665.703
Description    proFBSUP-ws
Name: 3, dtype: object

This next shot is still a profile with the name FBSUP, but the `ws` indicates it is a water surface shot. During parsing, any shots that are not a substrate shot (thalweg, riffle, run, pool, glide) are "associated" with the most recent substrate shot, meaning they will take on the northing and easting of that substrate shot. Thalweg calls should be made for profile shots if it is unclear whether or not the substrate is a riffle, run, pool or glide, and profile shots are required to have at least one valid indicator. Cross section shots do not need an indicator but it is assumed it is a substrate shot if nothing is provided.

Some care must be taken not to inadvertently mark shots with erroneous indicators. For example, `proR1-thw-strinvert` will be marked as three things: a thalweg shot, the location of a structure and a riffle. However, this description is probably meant to indicate the location of an invert, and would be better expressed as `proR1-thw-str_invert`.

The csv will be filtered using each name/morphtype combo found before the first break character. Each reach or cross section does not need to be continuous in the survey (e.g., a profile can be broken up by a cross section and vice-versa) but profiles do need to be ordered from upstream to downstream and cross sections are generally collected left to right (when facing downstream).

You can inspect what profiles and cross sections have been identified:

In [25]:
survey.get_names()

{'Profiles': {'proFBSUP': 121,
  'proFB': 90,
  'proR1': 133,
  'proCC': 141,
  'proR3': 157,
  'proR2': 164},
 'Cross Sections': {'xsFBSUPpo': 49,
  'xsFBSUPri': 45,
  'xsFBpo': 35,
  'xsFBri': 28,
  'xsR1po': 58,
  'xsR1ri': 52,
  'xsR1SUP': 38,
  'xsCCpo': 49,
  'xsCCri': 38,
  'xsR3ri': 49,
  'xsR3po': 73,
  'xsR3sup': 72,
  'xsR2ri': 32,
  'xsR2po': 53}}

This is a nested dictionary with two top level keys, `Profiles` and `Cross Sections`. Each key points to another dictionary where each key is a feature name that points to the number of shots that are within it. Note that if the description does not indicate either a profile or cross section *before the first break character* then it will be excluded entirely during parsing. If you're satisfied all your features have been properly identified, you can use the the `get_profile_objects()` and `get_cross_objects()` methods to get a list of Profile and CrossSection objects respectively.

In [23]:
profiles = survey.get_profile_objects()
crosses = survey.get_cross_objects()

Depending on your survey data you may get some warnings about some cross sections or profiles.

## Custom imports

The default parameters when initializing a StreamSurvey object are as follows:

`StreamSurvey(file, sep=',', metric=False, keywords=None, colRelations=None)`

The sep argument indicates what delimiter is used in the csv (generally a comma). By default surveys are assumed to be done in feet, but if `metric` is set to `True` then the units will be assumed to be meters. The `keywords` and `colRelations` are `None` by default and standard dictionaries are passed to them upon intialization unless they are passed something else. `colRelations` takes a dictionary where each key is a standardized string that maps onto the column names in your csv.

In [28]:
survey.colRelations

{'shotnum': 'Name',
 'whys': 'Northing',
 'exes': 'Easting',
 'zees': 'Elevation',
 'desc': 'Description'}

Above is the standard `colRelations` dict. They keys shown are all mandatory, but you can alter the associated values to fit your csv. However, it is recommended to use the standard dictionary when possible.

`keywords` is used to parse the description column in your survey.

In [29]:
survey.keywords

{'Profile': 'pro',
 'Thalweg': 'thw',
 'Riffle': 'ri',
 'Run': 'ru',
 'Pool': 'po',
 'Glide': 'gl',
 'Water Surface': 'ws',
 'Bankfull': 'bkf',
 'Top of Bank': 'tob',
 'Cross Section': 'xs',
 'Structure': 'str',
 'breakChar': '-',
 'commentChar': '_'}

Like `colRelations`, `keywords` is a dict. Not all keys are mandatory, but `Profile`, `Cross Section`, `Thalweg`, `breakChar` and `commentChar` are. Additionally, `Riffle`, `Run`, `Pool`, `Glide`, `Water Surface`, `Bankfull` and `Top of Bank` are recommended as they are used to fill out some fields both `Profile` and `CrossSection` objects. Any additional keys do not effect the produced `Cross Section` objects but they do create new columns in the `df` attribute for the produced `Profile` objects.

The methods `get_profile_objects()` and `get_cross_objects()` also have some default parameters:

`survey.get_profile_objects(stripName=False)`

`survey.get_profile_objects(guessType=True, project=True, stripName=False)`

For both methods the `stripName` argument, when `True`, removes the values paired to the `Profile` and `Cross Section` keys in the `keywords` dictionary from the parameters passed as the `name` argument to each `Profile` or `CrossSection` object created. For example, in our variable `survey`, `proFBSUP` would be stripped to `FBSUP`. If `stripName` is `False`, then the name passed would be `proFBSUP`.

`project` indicates whether or not the cross sections should have their survey shots projected onto the centerline connecting the first and last points in each cross section, and if `guessType` is `True` then `StreamSurvey` will attempt to guess if it's a riffle, run, pool or glide. It does this by looking if the name/morphtype combo (e.g., `xsFBSUPpo`) contains a value paired to any of the keys `Riffle`, `Run`, `Pool` or `Glide` in `keywords`. If `guessType` is `True`, care must be taken to not accidently mark a morphology in the name/morphtype combo. For example if you're survey a cross section on the piney river and your description is `xsPineyRiver`, the ri in river would be incorrectly identified as a riffle is `guessType` is `True` and the default `keywords` dictionary is used. In instances like this then either `guessType` should be set to `False`, the description changed, or the `keywords` dictionary altered.

In [31]:
crosses[1].df

,exes,whys,zees,desc,Station
0,1117.873323,1032.184436,668.685738,xsFBSUPri-lbp,0.000000
1,1117.930022,1032.060818,668.304030,xsFBSUPri,0.118115
2,1117.798719,1030.516229,668.278534,xsFBSUPri,1.668264
3,1117.660277,1027.880475,668.181614,xsFBSUPri,4.305927
4,1117.398727,1025.626678,668.245860,xsFBSUPri,6.574028
5,1117.272372,1023.152709,668.250774,xsFBSUPri,9.049472
6,1117.037847,1020.540625,668.187817,xsFBSUPri,11.672062
7,1116.909199,1018.230017,668.033681,xsFBSUPri,13.984989
8,1116.727023,1016.597222,667.974555,xsFBSUPri,15.627500
9,1116.623377,1014.620962,667.607253,xsFBSUPri,17.605178
